In [32]:
import spacy

# Load spaCy's Latin model
nlp = spacy.load("la_core_web_lg")

# Tokenize and keep both tokens and positions
def tokenize_with_offsets(text):
    doc = nlp(text)
    tokens = [token.text for token in doc]
    offsets = [(token.idx, token.idx + len(token)) for token in doc]
    return tokens, offsets

# Create sliding windows
def token_windows(tokens, offsets, n):
    return [
        (tokens[i:i+n], offsets[i:i+n])
        for i in range(len(tokens) - n + 1)
    ]

# Jaccard similarity between token sets
def jaccard(set1, set2):
    intersection = set1 & set2
    union = set1 | set2
    return len(intersection) / len(union) if union else 0

# Main matcher function with merging and char span output
def find_merged_spacy_matches(s1, s2, window_size=20, threshold=0.5, gap=10):
    tokens1, offsets1 = tokenize_with_offsets(s1)
    tokens2, offsets2 = tokenize_with_offsets(s2)

    win1 = token_windows(tokens1, offsets1, window_size)
    win2 = token_windows(tokens2, offsets2, window_size)

    raw_matches = []
    for i, (t1, o1) in enumerate(win1):
        for j, (t2, o2) in enumerate(win2):
            score = jaccard(set(t1), set(t2))
            if score >= threshold:
                raw_matches.append((score, i, i+window_size, j, j+window_size))

    # Merge clusters of overlapping/adjacent windows
    clusters = []
    current = []

    for match in sorted(raw_matches, key=lambda x: (x[1], x[3])):
        _, s1_start, s1_end, s2_start, s2_end = match
        if not current:
            current.append(match)
        else:
            _, ps1_start, ps1_end, ps2_start, ps2_end = current[-1]
            if s1_start <= ps1_end + gap and s2_start <= ps2_end + gap:
                current.append(match)
            else:
                clusters.append(current)
                current = [match]
    if current:
        clusters.append(current)

    # Final output: merged tokens and character spans
    merged = []
    for cluster in clusters:
        scores = [m[0] for m in cluster]
        s1_start = min(m[1] for m in cluster)
        s1_end = max(m[2] for m in cluster)
        s2_start = min(m[3] for m in cluster)
        s2_end = max(m[4] for m in cluster)

        tokens_span1 = tokens1[s1_start:s1_end]
        tokens_span2 = tokens2[s2_start:s2_end]

        char_start1 = offsets1[s1_start][0]
        char_end1 = offsets1[s1_end-1][1]
        char_start2 = offsets2[s2_start][0]
        char_end2 = offsets2[s2_end-1][1]

        merged.append({
            "score": max(scores),
            "tokens1": tokens_span1,
            "tokens2": tokens_span2,
            "char_span1": (char_start1, char_end1),
            "char_span2": (char_start2, char_end2)
        })

    return merged


In [33]:
sentence1 = (
    "nunc cognoui quod timeas Dominum, et non pepercisti unigenito filio tuo propter ne."
)

sentence2 = (
    "Ebraica magis sunt elegantia, Ne extendas manum tuam super puerum, nec feceris ei "
    "quicquam, quoniam nunc cognoui quod times tu deum, quod non pepercisti filio tuo unico a me."
)

matches = find_merged_spacy_matches(sentence1, sentence2, window_size=5, threshold=0.6)

for i, match in enumerate(matches, 1):
    print(f"\nMatch {i} (Jaccard {match['score']:.3f}):")
    print("  Sentence 1:", " ".join(match["tokens1"]))
    print("  Sentence 2:", " ".join(match["tokens2"]))
    print("  Char span in s1:", match["char_span1"])
    print("  Char span in s2:", match["char_span2"])
    print("  → s1 excerpt:", sentence1[match['char_span1'][0]:match['char_span1'][1]])
    print("  → s2 excerpt:", sentence2[match['char_span2'][0]:match['char_span2'][1]])



Match 1 (Jaccard 0.667):
  Sentence 1: non pepercisti unigenito filio tuo
  Sentence 2: quod non pepercisti filio tuo unico
  Char span in s1: (37, 71)
  Char span in s2: (133, 168)
  → s1 excerpt: non pepercisti unigenito filio tuo
  → s2 excerpt: quod non pepercisti filio tuo unico


In [2]:
from rapidfuzz import fuzz
import spacy

nlp = spacy.load("la_core_web_lg")

def fuzzy_match_spans(s1, s2, window_size=6, threshold=80):
    doc1 = nlp(s1)
    doc2 = nlp(s2)

    tokens1 = [token.text for token in doc1]
    tokens2 = [token.text for token in doc2]

    results = []

    for i in range(len(tokens1) - window_size + 1):
        span1 = tokens1[i:i+window_size]
        str1 = " ".join(span1)
        for j in range(len(tokens2) - window_size + 1):
            span2 = tokens2[j:j+window_size]
            str2 = " ".join(span2)
            score = fuzz.partial_ratio(str1, str2)
            if score >= threshold:
                # Get char offsets
                start1 = doc1[i].idx
                end1 = doc1[i + window_size - 1].idx + len(doc1[i + window_size - 1])
                start2 = doc2[j].idx
                end2 = doc2[j + window_size - 1].idx + len(doc2[j + window_size - 1])

                results.append({
                    "score": score,
                    "s1_tokens": span1,
                    "s2_tokens": span2,
                    "s1_text": s1[start1:end1],
                    "s2_text": s2[start2:end2],
                    "s1_span": (start1, end1),
                    "s2_span": (start2, end2)
                })

    # Sort by similarity
    return sorted(results, key=lambda x: x["score"], reverse=True)


In [7]:
s1 = "nunc cognoui quod timeas Dominum, et non pepercisti unigenito filio tuo propter ne."
s2 = "Ebraica magis sunt elegantia, Ne extendas manum tuam super puerum, nec feceris ei quicquam, quoniam nunc cognoui quod times tu deum, quod non pepercisti filio tuo unico a me."

matches = fuzzy_match_spans(s1, s2, window_size=15, threshold=70)

for i, match in enumerate(matches[:3], 1):
    print(f"\nMatch {i} (score {match['score']}):")
    print("  s1:", match['s1_text'])
    print("  s2:", match['s2_text'])



Match 1 (score 74.82993197278911):
  s1: nunc cognoui quod timeas Dominum, et non pepercisti unigenito filio tuo propter ne.
  s2: nunc cognoui quod times tu deum, quod non pepercisti filio tuo unico a me

Match 2 (score 72.36842105263158):
  s1: nunc cognoui quod timeas Dominum, et non pepercisti unigenito filio tuo propter ne.
  s2: , quoniam nunc cognoui quod times tu deum, quod non pepercisti filio tuo unico

Match 3 (score 72.36842105263158):
  s1: nunc cognoui quod timeas Dominum, et non pepercisti unigenito filio tuo propter ne.
  s2: quoniam nunc cognoui quod times tu deum, quod non pepercisti filio tuo unico a


In [72]:
s1 = "descendit in terram, induit hominem, et rursus ascendit in caelum, ut omnia impleret. Angeli quoque Dei ascendunt et descendunt"
s2 = "Secundo, quia Christus non magis in coelo, quam in terra est Dominus et gubernator. Ascendunt et descendum Angeli in hac scala."

matches = fuzzy_match_spans(s1, s2, window_size=15, threshold=80)

for i, match in enumerate(matches[:3], 1):
    print(f"\nMatch {i} (score {match['score']}):")
    print("  s1:", match['s1_text'])
    print("  s2:", match['s2_text'])


In [70]:
s1 = "Unus est qui baptizat. Unus et solus hic est, qui sanctificat, unus est fons, unus est riuus baptismi salutaris, qui per qualescunque canales influat: Hic est qui baptizat, de cuius ore profluit ipsa uis sacramenti, purificans, emundans, uiuificans totum fidelem hominem et sanctificans."
s2 = "Unus siquidem est, qui baptizat: unus et solus, qui sanctificat. cumque canales influat, hic est qui baptizat, de cuius ore profluit ipsa uis Sacramenti purificans, emundans, uiuificans totum fidelem hominem."
matches = fuzzy_match_spans(s1, s2, window_size=15, threshold=80)

for i, match in enumerate(matches[:3], 1):
    print(f"\nMatch {i} (score {match['score']}):")
    print("  s1:", match['s1_text'])
    print("  s2:", match['s2_text'])



Match 1 (score 97.91666666666666):
  s1: cuius ore profluit ipsa uis sacramenti, purificans, emundans, uiuificans totum fidelem hominem
  s2: de cuius ore profluit ipsa uis Sacramenti purificans, emundans, uiuificans totum fidelem hominem

Match 2 (score 97.91666666666666):
  s1: cuius ore profluit ipsa uis sacramenti, purificans, emundans, uiuificans totum fidelem hominem
  s2: cuius ore profluit ipsa uis Sacramenti purificans, emundans, uiuificans totum fidelem hominem.

Match 3 (score 97.8021978021978):
  s1: de cuius ore profluit ipsa uis sacramenti, purificans, emundans, uiuificans totum fidelem
  s2: , de cuius ore profluit ipsa uis Sacramenti purificans, emundans, uiuificans totum fidelem


In [71]:

s1 = "Fuit homo missus a Deo, cui nomen erat Joannes, hic uenit in testimonium, siquidem et alii prophetae homines fuerunt et omnes a Deo missi sunt, et nomine Joannis multi homines [Col.0214D] aequeuocati sunt. Diligentius ergo pensanda sunt, quia non exili elocutione, sed grandi declamatione prolata sunt. Uidelicet tali in loco uel causa sic nominatus homo fuit, multum suscipiendus, longe dissimilis hominibus multis, utpote quo maior inter natos mulierum nemo surrexit, adeo magnus, ut longe ante per prophetam, angelus dictus sit."
s2= "Congrue autem hominem eum nominat, fuit enim hoc nomine dignissimus, ut qui non fuit ex eorum numero, qui cum in honore essent, non intelligentes, comparati sunt iumentis insipientibus. Homo igitur erat Ioannes, sed multum suscipiendus, longe dissimilis hominibus aliis, utpote, quo maior non surrexit adeo magnus, ut longe ante per prophetam angelus dictus sit"
matches = fuzzy_match_spans(s1, s2, window_size=15, threshold=80)

for i, match in enumerate(matches[:3], 1):
    print(f"\nMatch {i} (score {match['score']}):")
    print("  s1:", match['s1_text'])
    print("  s2:", match['s2_text'])



Match 1 (score 95.83333333333334):
  s1: surrexit, adeo magnus, ut longe ante per prophetam, angelus dictus sit.
  s2: quo maior non surrexit adeo magnus, ut longe ante per prophetam angelus dictus sit

Match 2 (score 95.36423841059603):
  s1: nemo surrexit, adeo magnus, ut longe ante per prophetam, angelus dictus sit
  s2: quo maior non surrexit adeo magnus, ut longe ante per prophetam angelus dictus sit

Match 3 (score 92.85714285714286):
  s1: surrexit, adeo magnus, ut longe ante per prophetam, angelus dictus sit.
  s2: , quo maior non surrexit adeo magnus, ut longe ante per prophetam angelus dictus


In [82]:

s1 = "Fuit homo missus a Deo, cui nomen erat Joannes, hic uenit in testimonium, siquidem et alii prophetae homines fuerunt et omnes a Deo missi sunt, et nomine Joannis multi homines aequeuocati sunt. Diligentius ergo pensanda sunt, quia non exili elocutione, sed grandi declamatione prolata sunt. Uidelicet tali in loco uel causa sic nominatus homo fuit, multum suscipiendus, longe dissimilis hominibus multis, utpote quo maior inter natos mulierum nemo surrexit, adeo magnus, ut longe ante per prophetam, angelus dictus sit."
s2 = "Congrue autem hominem eum nominat, fuit enim hoc nomine dignissimus, ut qui non fuit ex eorum numero, qui cum in honore essent, non intelligentes, comparati sunt iumentis insipientibus. Homo igitur erat Ioannes, sed multum suscipiendus, longe dissimilis hominibus aliis, utpote, quo maior non surrexit adeo magnus, ut longe ante per prophetam angelus dictus sit"
matches = fuzzy_match_spans(s1, s2, window_size=15, threshold=70)

for i, match in enumerate(matches[:3], 1):
    print(f"\nMatch {i} (score {match['score']}):")
    print("  s1:", match['s1_text'])
    print("  s2:", match['s2_text'])



Match 1 (score 95.83333333333334):
  s1: surrexit, adeo magnus, ut longe ante per prophetam, angelus dictus sit.
  s2: quo maior non surrexit adeo magnus, ut longe ante per prophetam angelus dictus sit

Match 2 (score 95.36423841059603):
  s1: nemo surrexit, adeo magnus, ut longe ante per prophetam, angelus dictus sit
  s2: quo maior non surrexit adeo magnus, ut longe ante per prophetam angelus dictus sit

Match 3 (score 92.85714285714286):
  s1: surrexit, adeo magnus, ut longe ante per prophetam, angelus dictus sit.
  s2: , quo maior non surrexit adeo magnus, ut longe ante per prophetam angelus dictus


In [85]:

s1= "Ubi enim ulla, quae maioris pretii sit, reperiri potest? Hanc animarum uerus amator Deus, dilectae Joannis animae, pro monumento dilectionis praecipuae, fixit in pectore. Ut Uerbum, quod Maria Uirgo sola protulit in carne, ipsum huius socia uirginitas, prae omnibus sanctis, uiua mortalibus promeret uoce. Danda ergo sunt omnia, ut haec sola margarita comparari queat, omnesque carnalium sordes affectuum, ab oculis cordis abstergendae sunt eis, qui in schola Christi uenerabilibus student litteris: ut hanc aliquatenus ualeant aquilam prosequi, quam cordis munditia iuuit, ut claritatem solis aeterni plus caeteris diuinae uisionis [Col.0205D] animalibus, irreuerberata possint mentis acie contemplari. Nam de eo, qui per munditiae uiam ad ueram tendit sapientiam, loquitur Dominus per Isaiam: Iste in excelsis habitabit, munimenta saxorum sublimitas eius, panis ei datus est, aquae eius fidelis sunt. Regem in decore suo uidebunt oculi eius, cernent terram de longe (Isa. XXXIII). Imo, quod ad rem euidentius attinet, hic, sicut ad beatum Job, aliis quidem uerbis, sed eodem sensu dictum est: Ad praeceptum Domini eleuabitur, ut aquila in arduis ponet nidum suum, in petris manet, et in praeruptis silicibus commoratur atque inaccessis rupibus. Inde contemplatur escam, oculi eius de longe prospiciunt (Job XXXIX). Et quidem, haec omnia Joannes, huius Uerbi ac sempiterni principii contemplator excelsus, magnifice assecutus est. Quia uidelicet eleuatus est ut aquila, apertos intendens oculos in Diuinitatis radios, et in arduis posuit nidum suum, id est, aeterni huius Euangelii sui munimentum in petris mansit, id est, in soliditate ueritatis. Atque inde contemplatus est escam, scilicet illam quam et attingere meruit, huius Uerbi sempiterni gloriam."
s2= "Praeterea uides hic, quam merito Sanctus ille Ioannes inter alia animalia, aquilae comparetur. Aquilae enim conditiones sunt, quod supra alias aues uolat, in excelsis ponit nidum suum, inde contemplatur escam"

matches = fuzzy_match_spans(s1, s2, window_size=15, threshold=70)

for i, match in enumerate(matches[:3], 1):
    print(f"\nMatch {i} (score {match['score']}):")
    print("  s1:", match['s1_text'])
    print("  s2:", match['s2_text'])


In [8]:

s1="sy hand einen pundt miteinander yngeschnitten, quemadmodum apud nos schedas excindimus."
s2="In Hebraeo habetur exciderunt foedus, quemadmodum apud nos schaedas excindimus."
matches = fuzzy_match_spans(s1, s2, window_size=15, threshold=70)

for i, match in enumerate(matches[:3], 1):
    print(f"\nMatch {i} (score {match['score']}):")
    print("  s1:", match['s1_text'])
    print("  s2:", match['s2_text'])


In [67]:
from rapidfuzz import fuzz
import spacy

nlp = spacy.load("la_core_web_lg")

def get_dynamic_window_bounds(len1, len2, min_limit=10, max_limit=20):
    avg_len = (len1 + len2) / 2
    min_window = max(min_limit, int(avg_len * 0.2))
    max_window = min(max_limit, int(avg_len * 0.6))
    return min_window, max_window


def fuzzy_match_spans_dynamic(s1, s2, threshold=80):
    doc1 = nlp(s1)
    doc2 = nlp(s2)

    tokens1 = [token.lemma_ for token in doc1 if not token.is_punct]
    tokens2 = [token.lemma_ for token in doc2 if not token.is_punct]

    min_window, max_window = get_dynamic_window_bounds(len(tokens1), len(tokens2))

    results = []

    for window_size in range(min_window, max_window + 1):
        for i in range(len(tokens1) - window_size + 1):
            span1 = tokens1[i:i+window_size]
            str1 = " ".join(span1)

            for j in range(len(tokens2) - window_size + 1):
                span2 = tokens2[j:j+window_size]
                str2 = " ".join(span2)

                score = fuzz.token_set_ratio(str1, str2)
                adjusted_score = score * (window_size / max_window)

                if adjusted_score >= threshold:
                    start1 = doc1[i].idx
                    end1 = doc1[i + window_size - 1].idx + \
                        len(doc1[i + window_size - 1])
                    start2 = doc2[j].idx
                    end2 = doc2[j + window_size - 1].idx + \
                        len(doc2[j + window_size - 1])

                    results.append({
                        "score": score,
                        "s1_tokens": span1,
                        "s2_tokens": span2,
                        "s1_text": s1[start1:end1],
                        "s2_text": s2[start2:end2],
                        "s1_span": (start1, end1),
                        "s2_span": (start2, end2)
                    })

    return sorted(results, key=lambda x: x["score"], reverse=True)


In [68]:

s1="Utrumque datum donum huius baptismi, hic idem Joannes Baptista apud euangelistas Matthaeum et Lucam exprimit, cum dicit: Ipse uos baptizabit in Spiritu sancto et igni. Baptizat enim nos Spiritu sancto, cum in fontem baptismi descendente inuisibili gratia eiusdem Spiritus sancti omnia peccata eorum, qui baptizantur, dimittit."
s2="Huc pertinet, quod Ioannes apud Matthaeum dixit. Ipse baptizabit Spiritu sancto, et igni. Spiritu sancto baptizat, cum peccata remittit:"
matches = fuzzy_match_spans_dynamic(s1, s2)

for i, match in enumerate(matches[:3], 1):
    print(f"\nMatch {i} (score {match['score']}):")
    print("  s1:", match['s1_text'])
    print("  s2:", match['s2_text'])

In [69]:

s1 = "nunc cognoui quod timeas Dominum, et non pepercisti unigenito filio tuo propter ne."
s2 = "Ebraica magis sunt elegantia, Ne extendas manum tuam super puerum, nec feceris ei quicquam, quoniam nunc cognoui quod times tu deum, quod non pepercisti filio tuo unico a me."

matches = fuzzy_match_spans_dynamic(s1, s2)

for i, match in enumerate(matches[:3], 1):
    print(f"\nMatch {i} (score {match['score']}):")
    print("  s1:", match['s1_text'])
    print("  s2:", match['s2_text'])


Match 1 (score 92.0):
  s1: nunc cognoui quod timeas Dominum, et non pepercisti unigenito filio
  s2: quicquam, quoniam nunc cognoui quod times tu deum, quod

Match 2 (score 85.98130841121495):
  s1: nunc cognoui quod timeas Dominum, et non pepercisti unigenito filio tuo
  s2: quicquam, quoniam nunc cognoui quod times tu deum, quod non

Match 3 (score 85.18518518518519):
  s1: nunc cognoui quod timeas Dominum, et non pepercisti unigenito filio tuo
  s2: ei quicquam, quoniam nunc cognoui quod times tu deum, quod


In [70]:

s1="Non ergo sic tentat, ut exploret ea, quae prius nescierit, sed fidem nostram tentat deus, ut nos erudiat, ut Abra hae fidem illustrem ac manifestam faciat nobisque huiusmodi exempla ad imitanda proponat."
s2="Tentauit autem Abrahamum Deus, non ut exploret ea quae prius nesciret, sed ut Abrahae fidem illustriorem faciat, nobisque huiusmodi exempla imitanda proponat."
matches = fuzzy_match_spans_dynamic(s1, s2)

for i, match in enumerate(matches[:3], 1):
    print(f"\nMatch {i} (score {match['score']}):")
    print("  s1:", match['s1_text'])
    print("  s2:", match['s2_text'])



Match 1 (score 82.64462809917356):
  s1: Non ergo sic tentat, ut exploret ea, quae prius nescierit, sed fidem nostram
  s2: Tentauit autem Abrahamum Deus, non ut exploret ea quae prius nesciret, sed ut Abrahae

Match 2 (score 80.0):
  s1: tentat, ut exploret ea, quae prius nescierit, sed fidem nostram tentat deus,
  s2: Tentauit autem Abrahamum Deus, non ut exploret ea quae prius nesciret, sed ut Abrahae


In [71]:

s1="Uel certe, ut iam secundum Scripturam aliam loquar, quod factum est, spectabilis atque deliciosus coram sapientiae suae Deo ludus erat. Sic enim dicit ipsa, quae est hoc Dei Uerbum, Sapientia"
s2="Uel aliter: Quod factum est, hoc est, ipsa opera Domini antequam fierent, iam erant uita ipsius uerbi, hoc est, quasi ludus quidam, in quo mire delectabatur, etiam antequam fierent, quemadmodum sapientia Dei dicit"
matches = fuzzy_match_spans_dynamic(s1, s2)

for i, match in enumerate(matches[:3], 1):
    print(f"\nMatch {i} (score {match['score']}):")
    print("  s1:", match['s1_text'])
    print("  s2:", match['s2_text'])


In [72]:

s1="Docet autem Moses, quod sicut per sese gigni res non potuere, ita nec per sese perfici, et ut antem de conditione diximus, ita nunc de rerum conseruatione et perfectione dicendum est."
s2="Docet autem Moses hoc uerbo, quod sicut res per sese gigni non potuerit, ita nec per sese perfici. Et sicut ante de conditione rerum dictum est, sic nunc de perfectione rerum dicendum et conseruatione."

matches = fuzzy_match_spans_dynamic(s1, s2)

for i, match in enumerate(matches[:3], 1):
    print(f"\nMatch {i} (score {match['score']}):")
    print("  s1:", match['s1_text'])
    print("  s2:", match['s2_text'])



Match 1 (score 96.93251533742331):
  s1: Docet autem Moses, quod sicut per sese gigni res non potuere, ita nec per sese perfici,
  s2: Docet autem Moses hoc uerbo, quod sicut res per sese gigni non potuerit, ita nec per sese

Match 2 (score 96.6887417218543):
  s1: autem Moses, quod sicut per sese gigni res non potuere, ita nec per sese perfici,
  s2: autem Moses hoc uerbo, quod sicut res per sese gigni non potuerit, ita nec per sese

Match 3 (score 96.6887417218543):
  s1: , quod sicut per sese gigni res non potuere, ita nec per sese perfici, et ut
  s2: , quod sicut res per sese gigni non potuerit, ita nec per sese perfici. Et sicut


In [73]:

s1="Quia uidelicet cum sit cursoris officium, uel tantillum exspectare donec uideat quid nuntiet eis ad quos recursurus est, illi prius hinc per uiam uniuersae carnis recurrerunt, quam praestolatum Domini uidere mererentur aduentum. "
s2="Primo, siquidem quia dies aliorum uelociores cursores fuerunt: quia prius uiam uniuersae carnis intrauerunt, quam praestolatū Domini mererentur uidere aduentum."

matches = fuzzy_match_spans_dynamic(s1, s2)

for i, match in enumerate(matches[:3], 1):
    print(f"\nMatch {i} (score {match['score']}):")
    print("  s1:", match['s1_text'])
    print("  s2:", match['s2_text'])



Match 1 (score 86.74698795180723):
  s1: recursurus est, illi prius hinc per uiam uniuersae carnis recurrerunt, quam praestolatum Domini
  s2: aliorum uelociores cursores fuerunt: quia prius uiam uniuersae carnis intrauerunt, quam praestolatū Domini

Match 2 (score 86.07594936708861):
  s1: est, illi prius hinc per uiam uniuersae carnis recurrerunt, quam praestolatum Domini
  s2: uelociores cursores fuerunt: quia prius uiam uniuersae carnis intrauerunt, quam praestolatū Domini

Match 3 (score 85.13513513513513):
  s1: quos recursurus est, illi prius hinc per uiam uniuersae carnis recurrerunt, quam praestolatum
  s2: dies aliorum uelociores cursores fuerunt: quia prius uiam uniuersae carnis intrauerunt, quam praestolatū


In [74]:

s1="Tenebrae hoc loco, sunt impii homines, et Tenebrae increduli, pro Hebraeorum phrasi, qua reliquiae pro reliquis, Saluatio pro saluatis, Perditio pro perditis"
s2= "Tenebras autem eos nominat Hebraica phrasi, qua reliquiae pro reliquis, saluatio pro saluatis, circumcisio pro circumcisis, frequenter ponitur."
matches = fuzzy_match_spans_dynamic(s1, s2)

for i, match in enumerate(matches[:3], 1):
    print(f"\nMatch {i} (score {match['score']}):")
    print("  s1:", match['s1_text'])
    print("  s2:", match['s2_text'])



Match 1 (score 89.70588235294117):
  s1: Tenebrae increduli, pro Hebraeorum phrasi, qua reliquiae pro reliquis
  s2: eos nominat Hebraica phrasi, qua reliquiae pro reliquis, saluatio

Match 2 (score 89.70588235294117):
  s1: increduli, pro Hebraeorum phrasi, qua reliquiae pro reliquis,
  s2: eos nominat Hebraica phrasi, qua reliquiae pro reliquis, saluatio

Match 3 (score 85.91549295774648):
  s1: Tenebrae increduli, pro Hebraeorum phrasi, qua reliquiae pro reliquis,
  s2: autem eos nominat Hebraica phrasi, qua reliquiae pro reliquis, saluatio
